## TFRecords
TFRecords其实是一种二进制文件，虽然它不如其他格式好理解，但是它能更好的利用内存，更方便复制和移动，并且不需要单独的标签文件。

TFRecords文件包含了`tf.train.Example`协议内存块(protocol buffer)(协议内存块包含了字段 Features)。我们可以写一段代码获取你的数据，将数据填入到Example协议内存块(protocol buffer)，将协议内存块序列化为一个字符串，并且通过`tf.python_io.TFRecordWriter`写入到TFRecords文件。

从TFRecords文件中读取数据， 可以使用`tf.TFRecordReader`的`tf.parse_single_example`解析器。这个操作可以将Example协议内存块(protocol buffer)解析为张量。

In [1]:
import os
import sys
from PIL import Image
import tensorflow as tf
import numpy as np

TFRecords中存储的是**tf.train.Example**。一个**tf.train.Example**中包含一个**tf.train.Features**，**tf.train.Features**里包含**feature**参数。最后，feature参数是一个字典，包含多个键（字符串）与值（**tf.train.Feature**）。**tf.train.Feature**对象里包含有一个FloatList,或ByteList，或Int64List。

In [17]:
# 表情文件训练集与交叉验证集目录
training_data_path = './data/training_data'
cv_data_path = './data/crossvalidation_data'

# 表情种类
emotion_classes = ['surprise', 'fear', 'happy', 'sadness', 'disgust', 'anger']

# 创建TFRecordWriter
train_writer = tf.python_io.TFRecordWriter('emotion_train.tfrecords')
cv_writer = tf.python_io.TFRecordWriter('emotion_cv.tfrecords')

for index, emotion_name in enumerate(emotion_classes):
    train_emotion_path = training_data_path + '/' + emotion_name
    
    if(os.path.isdir(train_emotion_path)):
        for img_name in os.listdir(train_emotion_path):
            img_path = train_emotion_path + '/' + img_name
            img = Image.open(img_path)
            img = img.resize((32, 32))
        
            # 将图片转为Bytes
            img_raw = img.tobytes()
            example = tf.train.Example(features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
            }))
            # 序列化为字符串并写入tfrecords中
            train_writer.write(example.SerializeToString())

for index, emotion_name in enumerate(emotion_classes):
    cv_emotion_path = cv_data_path + '/' + emotion_name
    if(os.path.isdir(cv_emotion_path)):
        for img_name in os.listdir(cv_emotion_path):
            img_path = cv_emotion_path + '/' + img_name
            img = Image.open(img_path)
            img = img.resize((32, 32))
        
            img_raw = img.tobytes()
            example = tf.train.Example(features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
            }))

            cv_writer.write(example.SerializeToString())
        
train_writer.close()
cv_writer.close()

./data/crossvalidation_data/surprise
./data/crossvalidation_data/fear
./data/crossvalidation_data/happy
./data/crossvalidation_data/sadness
./data/crossvalidation_data/disgust
./data/crossvalidation_data/anger
